In [1]:
print("hi")

hi


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from torchvision import models
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [19]:
transform = transforms.Compose([transforms.ToTensor(),
transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
transforms.RandomHorizontalFlip(),
transforms.RandomResizedCrop(64)
])


In [20]:

data_dir = './hymenoptera_data'
image_datasets = {}
image_datasets['train'] = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
image_datasets['val'] = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
train_loader = DataLoader(image_datasets['train'],batch_size=128,shuffle=True,num_workers=2)
val_loader = DataLoader(image_datasets['val'],batch_size=128,shuffle=False,num_workers=2)

In [23]:
model=models.resnet18(pretrained=True)

#weights='IMAGENET1K_V1'
#freeze all the layers intially
for param in model.parameters():
    param.requires_grad = False
#(fc): Linear(in_features=512, out_features=1000, bias=True)
#replace the final fully connnected layers
num_features = model.fc.in_features
model.fc=nn.Linear(num_features,2)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(),lr=0.001)

for epoch in range(10):
    for image,label in train_loader:
        image,label = image.to(device),label.to(device)
        output=model(image)
        loss=criterion(output,label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"epoch:{epoch+1}, loss:{loss.item()}")
        


epoch:1, loss:0.8855292201042175
epoch:2, loss:0.7754966020584106
epoch:3, loss:0.6833823919296265
epoch:4, loss:0.7078452706336975
epoch:5, loss:0.6688253879547119
epoch:6, loss:0.6135544180870056
epoch:7, loss:0.5998637080192566
epoch:8, loss:0.5433840751647949
epoch:9, loss:0.4688367545604706
epoch:10, loss:0.4789082705974579


In [25]:
correct=0.0
total=0.0
model.eval()
with torch.no_grad():
    for image,label in val_loader:
        image,label = image.to(device),label.to(device)
        output=model(image)
        max,predicted=torch.max(output,1)
        correct+= (predicted==label).sum().item()
        total+=label.size(0)
    print(f"Test accuracy: {(correct/total)*100}")  

Test accuracy: 73.20261437908496
